In [201]:
import pandas as pd
import numpy as np
import datetime 
import optuna 
import xgboost
import os

In [202]:
os.listdir('data/data')
X_test = pd.read_csv(os.path.join(*['data', 'data', 'X_test.csv']), index_col = 0).reset_index(drop = True)
X_train = pd.read_csv(os.path.join(*['data', 'data', 'X_train.csv']), index_col = 0).reset_index(drop = True)
Y_test = pd.read_csv(os.path.join(*['data', 'data', 'Y_test.csv']), index_col = 0).reset_index(drop = True)
Y_train = pd.read_csv(os.path.join(*['data', 'data', 'Y_train.csv']), index_col = 0).reset_index(drop = True)

In [203]:
print(X_test.shape)
print(X_train.shape)
print(Y_test.shape)
print(Y_train.shape)
X_train.head()

(11633, 498)
(46530, 498)
(11633, 1)
(46530, 1)


,appid,type,name,release_date,developers,publishers,required_age,is_free,languages,positive_ratings,...,Wholesome,Word Game,World War I,World War II,Wrestling,Zombies,e-sports,metacritic_score,discount_percent,Ratings
0,1014200,1,Markerboard Jungle: Frogs,2022-03-05,['Primordial Muse'],['Primordial Muse'],0,0,['English'],0,...,0,0,0,0,0,0,0,74.0,30.0,0.405836
1,306550,1,Runers,2014-09-02,['LGK Games'],['Mastertronic'],0,0,"['English', ' German', ' Russian', ' Japanese']",226,...,0,0,0,0,0,0,0,68.0,0.0,0.760184
2,1949720,1,Dandi World,2022-04-18,['Ruken Games'],['Trialis Studios'],0,0,"['English', ' Spanish - Spain', ' Spanish - La...",2,...,0,0,0,0,0,0,0,74.0,0.0,0.640795
3,747730,1,Operation Red Dragon,2017-12-15,['PBFP'],['PBFP'],0,0,['English'],4,...,0,0,0,0,0,0,0,74.0,0.0,0.625066
4,1819290,1,Spaceship Hunters,2022-01-31,['PIerre Nury'],['Pierre Nury'],0,0,"['English', ' French', ' Portuguese - Portugal...",3,...,0,0,0,0,0,0,0,74.0,50.0,0.670594


In [204]:
# X_train = X_train.drop_duplicates().reset_index(drop = True).copy()
# X_test = X_test.drop_duplicates().reset_index(drop = True).copy()

In [205]:
X_train['developers'] = X_train['developers'].apply(lambda x: eval(x))
X_train['publishers'] = X_train['publishers'].apply(lambda x: eval(x))
X_train['languages'] = X_train['languages'].apply(lambda x: eval(x))

### Feature Engineering

In [206]:
category_list = ['1980s', "1990's", '2.5D', '2D', '2D Fighter', '2D Platformer', '360 Video', '3D', '3D Fighter', '3D Platformer', '3D Vision', '4 Player Local', '4X', '6DOF', '8-bit Music', 'Abstract', 'Accounting', 'Action', 'Action RPG', 'Action RTS', 'Action Roguelike', 'Action-Adventure', 'Addictive', 'Adventure', 'Agriculture', 'Aliens', 'Alternate History', 'Ambient', 'America', 'Animation & Modeling', 'Anime', 'Arcade', 'Archery', 'Arena Shooter', 'Artificial Intelligence', 'Assassin', 'Asymmetric VR', 'Asynchronous Multiplayer', 'Atmospheric', 'Audio Production', 'Auto Battler', 'Automation', 'Automobile Sim', 'BMX', 'Base-Building', 'Baseball', 'Based On A Novel', 'Basketball', 'Battle Royale', "Beat 'em up", 'Beautiful', 'Benchmark', 'Bikes', 'Blood', 'Board Game', 'Boss Rush', 'Bowling', 'Boxing', 'Building', 'Bullet Hell', 'Bullet Time', 'CRPG', 'Capitalism', 'Captions available', 'Card Battler', 'Card Game', 'Cartoon', 'Cartoony', 'Casual', 'Cats', 'Character Action Game', 'Character Customization', 'Chess', 'Choices Matter', 'Choose Your Own Adventure', 'Cinematic', 'City Builder', 'Class-Based', 'Classic', 'Clicker', 'Co-op', 'Co-op Campaign', 'Coding', 'Cold War', 'Collectathon', 'Colony Sim', 'Colorful', 'Combat', 'Combat Racing', 'Comedy', 'Comic Book', 'Commentary available', 'Competitive', 'Conspiracy', 'Controller', 'Conversation', 'Cooking', 'Cozy', 'Crafting', 'Creature Collector', 'Cricket', 'Crime', 'Cross-Platform Multiplayer', 'Cult Classic', 'Cute', 'Cyberpunk', 'Cycling', 'Dark', 'Dark Comedy', 'Dark Fantasy', 'Dark Humor', 'Dating Sim', 'Deckbuilding', 'Demons', 'Design & Illustration', 'Destruction', 'Detective', 'Difficult', 'Dinosaurs', 'Diplomacy', 'Documentary', 'Dog', 'Dragons', 'Drama', 'Driving', 'Dungeon Crawler', 'Dungeons & Dragons', 'Dynamic Narration', 'Dystopian ', 'Early Access', 'Economy', 'Education', 'Electronic', 'Electronic Music', 'Emotional', 'Epic', 'Episodic', 'Escape Room', 'Experience', 'Experimental', 'Exploration', 'FMV', 'FPS', 'Faith', 'Family Friendly', 'Fantasy', 'Farming', 'Farming Sim', 'Fast-Paced', 'Female Protagonist', 'Fighting', 'First-Person', 'Fishing', 'Flight', 'Football (American)', 'Football (Soccer)', 'Foreign', 'Free to Play', 'Full controller support', 'Funny', 'Futuristic', 'Gambling', 'Game Development', 'GameMaker', 'Games Workshop', 'Gaming', 'God Game', 'Golf', 'Gore', 'Gothic', 'Grand Strategy', 'Great Soundtrack', 'Grid-Based Movement', 'Gun Customization', 'Hack and Slash', 'Hacking', 'Hand-drawn', 'Heist', 'Hentai', 'Hero Shooter', 'Hex Grid', 'Hidden Object', 'Historical', 'Hockey', 'Horror', 'Horses', 'Hunting', 'Idler', 'Illuminati', 'Immersive', 'Immersive Sim', 'In-App Purchases', 'Includes Source SDK', 'Includes level editor', 'Indie', 'Instrumental Music', 'Intentionally Awkward Controls', 'Interactive Fiction', 'Inventory Management', 'Investigation', 'Isometric', 'JRPG', 'Jet', 'Job Simulator', 'Jump Scare', 'Kickstarter', 'LAN Co-op', 'LAN PvP', 'LEGO', 'LGBTQ+', 'Lemmings', 'Level Editor', 'Life Sim', 'Linear', 'Local Co-Op', 'Local Multiplayer', 'Logic', 'Loot', 'Looter Shooter', 'Lore-Rich', 'Lovecraftian', 'MMO', 'MMORPG', 'MOBA', 'Magic', 'Mahjong', 'Management', 'Mars', 'Martial Arts', 'Massively Multiplayer', 'Match 3', 'Mature', 'Mechs', 'Medical Sim', 'Medieval', 'Memes', 'Metroidvania', 'Military', 'Mini Golf', 'Minigames', 'Minimalist', 'Mining', 'Mod', 'Moddable', 'Modern', 'Mods (require HL2)', 'Motocross', 'Motorbike', 'Mouse only', 'Movie', 'Multi-player', 'Multiplayer', 'Multiple Endings', 'Music', 'Music-Based Procedural Generation', 'Musou', 'Mystery', 'Mystery Dungeon', 'Mythology', 'NSFW', 'Narration', 'Narrative', 'Nature', 'Naval', 'Naval Combat', 'Ninja', 'Noir', 'Nonlinear', 'Nostalgia', 'Nudity', 'Offroad', 'Old School', 'On-Rails Shooter', 'Online Co-Op', 'Online Co-op', 'Online PvP', 'Open World', 'Open World Survival Craft', 'Otome', 'Outbreak Sim', 'Parkour', 'Parody ', 'Partial Controller Support', 'Party', 'Party Game', 'Party-Based RPG', 'Perma Death', 'Philosophical', 'Photo Editing', 'Physics', 'Pinball', 'Pirates', 'Pixel Graphics', 'Platformer', 'Point & Click', 'Political', 'Political Sim', 'Politics', 'Pool', 'Post-apocalyptic', 'Precision Platformer', 'Procedural Generation', 'Programming', 'Psychedelic', 'Psychological', 'Psychological Horror', 'Puzzle', 'Puzzle-Platformer', 'PvE', 'PvP', 'Quick-Time Events', 'RPG', 'RPGMaker', 'RTS', 'Racing', 'Real Time Tactics', 'Real-Time', 'Real-Time with Pause', 'Realistic', 'Reboot', 'Relaxing', 'Remake', 'Remote Play Together', 'Remote Play on Phone', 'Remote Play on TV', 'Remote Play on Tablet', 'Replay Value', 'Resource Management', 'Retro', 'Rhythm', 'Robots', 'Rock Music', 'Rogue-like', 'Rogue-lite', 'Roguelike Deckbuilder', 'Roguevania', 'Romance', 'Rome', 'Rugby', 'Runner', 'Sailing', 'Sandbox', 'Satire', 'Sci-fi', 'Science', 'Score Attack', 'Sexual Content', 'Shared/Split Screen', 'Shared/Split Screen Co-op', 'Shared/Split Screen PvP', "Shoot 'Em Up", 'Shooter', 'Shop Keeper', 'Short', 'Side Scroller', 'Silent Protagonist', 'Simulation', 'Single-player', 'Singleplayer', 'Skateboarding', 'Skating', 'Skiing', 'Sniper', 'Snooker', 'Snow', 'Snowboarding', 'Social Deduction', 'Software', 'Software Training', 'Sokoban', 'Solitaire', 'Solo RV', 'Souls-like', 'Soundtrack', 'Space', 'Space Sim', 'Spaceships', 'Spectacle fighter', 'Spelling', 'Split Screen', 'Sports', 'Stats', 'Stealth', 'Steam Achievements', 'Steam Cloud', 'Steam Leaderboards', 'Steam Trading Cards', 'Steam Turn Notifications', 'Steam Workshop', 'SteamVR Collectibles', 'Steampunk', 'Story Rich', 'Strategy', 'Strategy RPG', 'Stylized', 'Submarine', 'Superhero', 'Supernatural', 'Surreal', 'Survival', 'Survival Horror', 'Swordplay', 'Tabletop', 'Tactical', 'Tactical RPG', 'Tanks', 'Team-Based', 'Tennis', 'Text-Based', 'Third Person', 'Third-Person Shooter', 'Thriller', 'Tile-Matching', 'Time Attack', 'Time Management', 'Time Manipulation', 'Time Travel', 'Top-Down', 'Top-Down Shooter', 'Touch-Friendly', 'Tower Defense', 'TrackIR', 'Tracked Controller Support', 'Trading', 'Trading Card Game', 'Traditional Roguelike', 'Trains', 'Transhumanism', 'Transportation', 'Trivia', 'Turn-Based', 'Turn-Based Combat', 'Turn-Based Strategy', 'Turn-Based Tactics', 'Tutorial', 'Twin Stick Shooter', 'Typing', 'Underground', 'Underwater', 'Unforgiving', 'Utilities', 'VR', 'VR Only', 'VR Support', 'VR Supported', 'Valve Anti-Cheat enabled', 'Vampire', 'Vehicular Combat', 'Video Production', 'Vikings', 'Villain Protagonist', 'Violent', 'Visual Novel', 'Voice Control', 'Volleyball', 'Voxel', 'Walking Simulator', 'War', 'Wargame', 'Warhammer 40K', 'Web Publishing', 'Werewolves', 'Western', 'Wholesome', 'Word Game', 'World War I', 'World War II', 'Wrestling', 'Zombies', 'e-sports',]

In [207]:
#Count the number of categories in a game 
X_train['num_categories'] = X_train[category_list].sum(axis = 1)
X_test['num_categories'] = X_test[category_list].sum(axis = 1)

In [208]:
#Count number of languages in a game 
X_train['num_languages'] = X_train['languages'].apply(len)
X_test['num_languages'] = X_test['languages'].apply(len)

In [209]:
#Count number of developers and publishers
X_train['num_developers'] = X_train['developers'].apply(len)
X_train['num_publishers'] = X_train['publishers'].apply(len)
X_test['num_developers'] = X_test['developers'].apply(len)
X_test['num_publishers'] = X_test['publishers'].apply(len)

In [210]:
#One hot encode languages
languages = {}
for avail_languages in X_train['languages']:
    for lang in avail_languages:
        lang = lang.strip()
        if lang in languages:
            languages[lang] += 1
        else:
            languages[lang] = 1
languages

{'English': 46097,
 'German': 11231,
 'Russian': 10410,
 'Japanese': 8688,
 'Spanish - Spain': 9840,
 'Spanish - Latin America': 2318,
 'French': 10796,
 'Portuguese - Portugal': 3258,
 'Portuguese - Brazil': 5502,
 'Italian': 7425,
 'Not supported': 1119,
 'Simplified Chinese': 10675,
 'Korean': 5429,
 'Traditional Chinese': 5213,
 'Polish': 4264,
 'Hungarian': 1542,
 'Dutch': 2493,
 'Czech': 1874,
 'Belarusian': 31,
 'Serbian': 20,
 'Slovak': 52,
 'Ukrainian': 1650,
 'Greek': 1196,
 'Swedish': 1657,
 'Thai': 1426,
 'Danish': 1372,
 'Turkish': 2979,
 'Norwegian': 1318,
 'Finnish': 1328,
 'Arabic': 1490,
 'Bulgarian': 1054,
 'Romanian': 1209,
 'Hindi': 29,
 'Vietnamese': 944,
 'Filipino': 10,
 'Indonesian': 67,
 'Malay': 20,
 'Catalan': 40,
 'Latvian': 12,
 'Afrikaans': 7,
 'Basque': 15,
 'Estonian': 9,
 'Hebrew': 25,
 'Icelandic': 5,
 'Slovenian': 6,
 'Lithuanian': 10,
 'Albanian': 7,
 'Bangla': 7,
 'Bosnian': 4,
 'Georgian': 5,
 'Persian': 13,
 'Urdu': 7,
 'Uzbek': 3,
 '#lang_#lang_#

In [211]:
del languages['#lang_#lang_#lang_english**#lang_full_audio*#lang_full_audio']
del languages['#lang_#lang_spanish*#lang_full_audio']
del languages['English[b]*[/b]']
del languages['French[b]*[/b]']
del languages['Italian[b]*[/b]']
del languages['German[b]*[/b]']
del languages['Spanish - Spain[b]*[/b]']
del languages['Korean[b]*[/b]']
del languages['Japanese \r\n\r\n[b]*[/b]']
del languages['Russian\r\n[b]*[/b]']
del languages['#lang_slovakian']
del languages['(all with full audio support)']
del languages[ 'English\r\nRussian\r\nSpanish - Spain\r\nJapanese\r\nCzech']

In [212]:
lang_dict_train = {}
for language in languages.keys():
    lang_dict_train[language] = []
    for avail_languages in X_train['languages']:
        if language in [x.strip() for x in avail_languages]:
            lang_dict_train[language].append(1)
        else:
            lang_dict_train[language].append(0)
            
lang_dict_test = {}
for language in languages.keys():
    lang_dict_test[language] = []
    for avail_languages in X_test['languages']:
        if language in [x.strip() for x in avail_languages]:
            lang_dict_test[language].append(1)
        else:
            lang_dict_test[language].append(0)

In [213]:
X_train = pd.concat([X_train, pd.DataFrame(lang_dict_train)], axis = 1)
X_test = pd.concat([X_test, pd.DataFrame(lang_dict_test)], axis = 1)
print(X_train.shape)
print(X_test.shape)

display(X_train.head())
display(X_test.head())

(46530, 575)
(11633, 575)


,appid,type,name,release_date,developers,publishers,required_age,is_free,languages,positive_ratings,...,Kazakh,Nepali,Mongolian,Khmer,Tamil,Macedonian,Scots,Dari,Punjabi (Gurmukhi),Telugu
0,1014200,1,Markerboard Jungle: Frogs,2022-03-05,[Primordial Muse],[Primordial Muse],0,0,[English],0,...,0,0,0,0,0,0,0,0,0,0
1,306550,1,Runers,2014-09-02,[LGK Games],[Mastertronic],0,0,"[English, German, Russian, Japanese]",226,...,0,0,0,0,0,0,0,0,0,0
2,1949720,1,Dandi World,2022-04-18,[Ruken Games],[Trialis Studios],0,0,"[English, Spanish - Spain, Spanish - Latin A...",2,...,0,0,0,0,0,0,0,0,0,0
3,747730,1,Operation Red Dragon,2017-12-15,[PBFP],[PBFP],0,0,[English],4,...,0,0,0,0,0,0,0,0,0,0
4,1819290,1,Spaceship Hunters,2022-01-31,[PIerre Nury],[Pierre Nury],0,0,"[English, French, Portuguese - Portugal, Po...",3,...,0,0,0,0,0,0,0,0,0,0


,appid,type,name,release_date,developers,publishers,required_age,is_free,languages,positive_ratings,...,Kazakh,Nepali,Mongolian,Khmer,Tamil,Macedonian,Scots,Dari,Punjabi (Gurmukhi),Telugu
0,525870,1,WackyMoles,2016-12-12,['CrystalGame'],['CrystalGame'],0,0,['English'],3,...,0,0,0,0,0,0,0,0,0,0
1,362350,1,Boss Monster,2016-04-27,['Plain Concepts Corp'],['Plain Concepts Corp'],0,0,['English'],223,...,0,0,0,0,0,0,0,0,0,0
2,1640,1,Disciples II: Gallean's Return,2006-07-06,['Strategy First'],['Strategy First'],0,0,['English'],1017,...,0,0,0,0,0,0,0,0,0,0
3,1480960,1,Godsbane,2022-03-04,['Project Hyperdrive'],['Project Hyperdrive'],0,1,"['English', ' German', ' French', ' Russian', ...",97,...,0,0,0,0,0,0,0,0,0,0
4,695810,1,HORDE ATTACK,2017-09-15,['Danil Kapylov'],['Danil Kapylov'],0,0,['English'],8,...,0,0,0,0,0,0,0,0,0,0


In [214]:
#One hot encode developers and publishers 
developers = {}
for devs in X_train['developers']:
    for dev in devs:
        dev = dev.strip()
        if dev in developers:
            developers[dev] += 1
        else:
            developers[dev] = 1
developers = pd.Series(developers).sort_values(ascending = False)

In [215]:
publishers = {}
for pubs in X_train['publishers']:
    for pub in pubs:
        pub = pub.strip()
        if pub in publishers:
            publishers[pub] += 1
        else:
            publishers[pub] = 1

publishers = pd.Series(publishers).sort_values(ascending = False)

In [216]:
print(X_train.shape)
print(X_test.shape)

(46530, 575)
(11633, 575)


In [217]:
#Take top 30 developers and publishers and do 1 hot encoding
top_30_devs = set(developers.iloc[0:30].index)
top_30_pubs = set(publishers.iloc[0:30].index)

dev_dict_train = {}
for top_dev in top_30_devs:
    dev_dict_train[f"{top_dev}_dev"] = []
    for devs in X_train['developers']:
        if top_dev in [x.strip() for x in devs]:
            dev_dict_train[f"{top_dev}_dev"].append(1)
        else:
            dev_dict_train[f"{top_dev}_dev"].append(0)
            
pub_dict_train = {}
for top_pub in top_30_pubs:
    pub_dict_train[f"{top_pub}_pub"] = []
    for pubs in X_train['publishers']:
        if top_pub in [x.strip() for x in pubs]:
            pub_dict_train[f"{top_pub}_pub"].append(1)
        else:
            pub_dict_train[f"{top_pub}_pub"].append(0)

dev_dict_test = {}
for top_dev in top_30_devs:
    dev_dict_test[f"{top_dev}_dev"] = []
    for devs in X_test['developers']:
        if top_dev in [x.strip() for x in devs]:
            dev_dict_test[f"{top_dev}_dev"].append(1)
        else:
            dev_dict_test[f"{top_dev}_dev"].append(0)
            
pub_dict_test = {}
for top_pub in top_30_pubs:
    pub_dict_test[f"{top_pub}_pub"] = []
    for pubs in X_test['publishers']:
        if top_pub in [x.strip() for x in pubs]:
            pub_dict_test[f"{top_pub}_pub"].append(1)
        else:
            pub_dict_test[f"{top_pub}_pub"].append(0)

In [218]:
X_train = pd.concat([X_train, pd.DataFrame(dev_dict_train), pd.DataFrame(pub_dict_train)], axis = 1)
X_test = pd.concat([X_test, pd.DataFrame(dev_dict_test), pd.DataFrame(pub_dict_test)], axis = 1)
print(X_train.shape)
print(X_test.shape)
display(X_train.head())
display(X_test.head())

(46530, 635)
(11633, 635)


,appid,type,name,release_date,developers,publishers,required_age,is_free,languages,positive_ratings,...,Sekai Project_pub,Alawar Entertainment_pub,Electronic Arts_pub,SEGA_pub,Strategy First_pub,Komodo_pub,Microids_pub,Slitherine Ltd._pub,Square Enix_pub,Hosted Games_pub
0,1014200,1,Markerboard Jungle: Frogs,2022-03-05,[Primordial Muse],[Primordial Muse],0,0,[English],0,...,0,0,0,0,0,0,0,0,0,0
1,306550,1,Runers,2014-09-02,[LGK Games],[Mastertronic],0,0,"[English, German, Russian, Japanese]",226,...,0,0,0,0,0,0,0,0,0,0
2,1949720,1,Dandi World,2022-04-18,[Ruken Games],[Trialis Studios],0,0,"[English, Spanish - Spain, Spanish - Latin A...",2,...,0,0,0,0,0,0,0,0,0,0
3,747730,1,Operation Red Dragon,2017-12-15,[PBFP],[PBFP],0,0,[English],4,...,0,0,0,0,0,0,0,0,0,0
4,1819290,1,Spaceship Hunters,2022-01-31,[PIerre Nury],[Pierre Nury],0,0,"[English, French, Portuguese - Portugal, Po...",3,...,0,0,0,0,0,0,0,0,0,0


,appid,type,name,release_date,developers,publishers,required_age,is_free,languages,positive_ratings,...,Sekai Project_pub,Alawar Entertainment_pub,Electronic Arts_pub,SEGA_pub,Strategy First_pub,Komodo_pub,Microids_pub,Slitherine Ltd._pub,Square Enix_pub,Hosted Games_pub
0,525870,1,WackyMoles,2016-12-12,['CrystalGame'],['CrystalGame'],0,0,['English'],3,...,0,0,0,0,0,0,0,0,0,0
1,362350,1,Boss Monster,2016-04-27,['Plain Concepts Corp'],['Plain Concepts Corp'],0,0,['English'],223,...,0,0,0,0,0,0,0,0,0,0
2,1640,1,Disciples II: Gallean's Return,2006-07-06,['Strategy First'],['Strategy First'],0,0,['English'],1017,...,0,0,0,0,0,0,0,0,0,0
3,1480960,1,Godsbane,2022-03-04,['Project Hyperdrive'],['Project Hyperdrive'],0,1,"['English', ' German', ' French', ' Russian', ...",97,...,0,0,0,0,0,0,0,0,0,0
4,695810,1,HORDE ATTACK,2017-09-15,['Danil Kapylov'],['Danil Kapylov'],0,0,['English'],8,...,0,0,0,0,0,0,0,0,0,0


In [219]:
X_train.drop(['appid', 'type', 'name', 'release_date', 'developers', 'publishers', 'languages'], axis = 1, inplace = True)
X_test.drop(['appid', 'type', 'name', 'release_date', 'developers', 'publishers', 'languages'], axis = 1, inplace = True)

### Modelling

In [220]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score

In [221]:
Y_train = Y_train.values.squeeze()
Y_test = Y_test.values.squeeze()

In [222]:
clf = RandomForestClassifier()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)

In [223]:
print(accuracy_score(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred, average = 'macro'))

0.7748646093011261
0.42837852938554055


In [227]:
feat_importances = pd.DataFrame(pd.Series(clf.feature_importances_, clf.feature_names_in_)).sort_values(0).head(20)
feat_importances

,0
Khmer,0.000000e+00
Instrumental Music,0.000000e+00
Punjabi (Gurmukhi),0.000000e+00
Luxembourgish,5.471298e-09
Tile-Matching,5.630375e-09
8-bit Music,7.936777e-09
Kazakh,8.513678e-09
Coding,1.517813e-08
Rock Music,3.748882e-08
Telugu,7.310484e-08


In [228]:
print(X_train['Khmer'].sum())
print(X_train['Punjabi (Gurmukhi)'].sum())
print(X_train['Scots'].sum())

1
1
1


Going to prune unimportant features first, there are too many columns in this dataset. Prune the one hot encode features that have too little values in them.

In [240]:
top_30_langs = set(pd.Series(languages).sort_values(ascending = False).iloc[0:30].index)

In [266]:
language_list = list(languages.keys())

In [242]:
categories_count = X_train[category_list].sum(axis = 0)
top_50_categories = set(categories_count.sort_values(ascending = False).iloc[0:50].index)

In [265]:
new_cols = [x for x in list(X_train.columns) if x not in set(set(category_list) - top_50_categories)]
new_cols = [x for x in list(new_cols) if x not in set(set(language_list) - top_30_langs)]
X_train = X_train.loc[:, new_cols]
X_test = X_test.loc[:, new_cols]

In [267]:
clf = RandomForestClassifier()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)

In [269]:
print(accuracy_score(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred, average = 'macro'))

0.7836327688472449
0.4758091858237284


In [268]:
feat_importances = pd.DataFrame(pd.Series(clf.feature_importances_, clf.feature_names_in_)).sort_values(0).head(20)
feat_importances

,0
windows,9.334472e-07
EpiXR Games UG_dev,3.403715e-06
AMAX Interactive_dev,5.877599e-06
Somer Games_dev,8.348070e-06
玫瑰工作室_dev,8.521736e-06
"Kairosoft Co.,Ltd_dev",1.174771e-05
Eipix Entertainment_dev,1.779391e-05
Cristian Manolachi_dev,2.238266e-05
Anamik Majumdar_dev,2.665071e-05
Atomic Fabrik_dev,2.679730e-05


Model performance remains the same, honestly can prune further features, most still seem quite irrelevant


### Hyperparameter tuning

Do hyperparameter tuning with cross validation

In [291]:
import lightgbm
from sklearn.model_selection import KFold
print(lightgbm.__version__)

3.3.5


In [286]:
import re
X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

Baseline Model

In [287]:
clf = lightgbm.LGBMClassifier(objective = 'multiclass')
clf.fit(X_temp, Y_train)
Y_pred = clf.predict(X_test)

In [288]:
print(accuracy_score(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred, average = 'macro'))

0.772457663543368
0.47092686705337583


In [295]:
Y_train = pd.Series(Y_train)
Y_test = pd.Series(Y_test)

##### LightGBM

In [298]:
def objective(trial):
    kf = KFold(n_splits=5, shuffle=False)
    accuracys = []
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 6, 64), 
        'max_depth': trial.suggest_int('max_depth', 1, 12),
        'n_estimators': trial.suggest_int('n_estimators', 1, 1000),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0,1),
    }
    for train_index, test_index in kf.split(X_train):
        cv_train_X, cv_train_Y = X_train.iloc[train_index], Y_train.iloc[train_index]
        cv_test_X, cv_test_Y = X_train.iloc[test_index], Y_train.iloc[test_index]
        lightgbm_clf = lightgbm.LGBMClassifier(objective = 'multiclass', **params)
        lightgbm_clf.fit(cv_train_X, cv_train_Y)
        cv_pred_Y = lightgbm_clf.predict(cv_test_X)
        accuracys.append(accuracy_score(cv_test_Y, cv_pred_Y))
    return np.mean(accuracys)
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials=20)

[I 2023-04-03 01:09:24,467] A new study created in memory with name: no-name-d979f3e9-db90-453f-95f9-68b3c57b2174
[I 2023-04-03 01:12:06,030] Trial 0 finished with value: 0.7885450247152376 and parameters: {'num_leaves': 47, 'max_depth': 9, 'n_estimators': 926, 'reg_alpha': 0.4318153635054113, 'reg_lambda': 0.014138088114799774}. Best is trial 0 with value: 0.7885450247152376.
[I 2023-04-03 01:12:53,006] Trial 1 finished with value: 0.7845046206748334 and parameters: {'num_leaves': 54, 'max_depth': 5, 'n_estimators': 390, 'reg_alpha': 0.8598777948071731, 'reg_lambda': 0.4919881358398981}. Best is trial 0 with value: 0.7885450247152376.
[I 2023-04-03 01:13:11,156] Trial 2 finished with value: 0.7830002149151086 and parameters: {'num_leaves': 30, 'max_depth': 4, 'n_estimators': 205, 'reg_alpha': 0.19800297213027496, 'reg_lambda': 0.05127002494613875}. Best is trial 0 with value: 0.7885450247152376.
[I 2023-04-03 01:15:38,285] Trial 3 finished with value: 0.7892327530625403 and parameters

KeyboardInterrupt: 

In [299]:
best_params = study.best_params
lightgbm_clf = lightgbm.LGBMClassifier(objective='multiclass', **best_params)
lightgbm_clf.fit(X_train, Y_train)
Y_pred = lightgbm_clf.predict(X_test)
print(f"Acc: {accuracy_score(Y_test, Y_pred)}")
print(f"F1: {f1_score(Y_test, Y_pred, average = 'macro')}")

Acc: 0.7799363878621164
F1: 0.4899600423871266
